In [ ]:
# inference.ipynb (500회 학습 버전)
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# 1. 데이터 및 폴더 준비
os.makedirs('data', exist_ok=True)
os.makedirs('model_artifacts', exist_ok=True)

# 2. 데이터 생성 (500개 이상)
formal_sentences = [
    "교수님, 이번 과제 제출 기한을 연장해주실 수 있으신가요?",
    "안녕하세요, 수업 내용 중 질문이 있어 메일 드립니다.",
    "출석 인정 관련하여 증빙 서류를 제출하고자 합니다.",
    "혹시 상담 가능하신 시간이 언제이신지 알려주시면 감사하겠습니다.",
    "졸업 요건과 관련하여 면담을 요청드려도 될까요?",
    "죄송하지만 개인적인 사정으로 수업 참석이 어렵습니다.",
    "팀 프로젝트 조 편성 문제로 문의드립니다."
] * 40

informal_sentences = [
    "야 오늘 점심 뭐 먹을래?",
    "아 진짜 과제 개많네 ㅋㅋㅋ",
    "롤 하러 갈 사람 구함",
    "너 어제 수업 들었냐? 필기 좀 보여줘",
    "오늘 술 한잔 ㄱㄱ?",
    "이번 주말에 시간 됨?",
    "아 교수님 진도 너무 빨리 나가시는 거 아니냐"
] * 40

data = [{'text': t, 'label': 1} for t in formal_sentences] + \
       [{'text': t, 'label': 0} for t in informal_sentences]

df = pd.DataFrame(data).sample(frac=1, random_state=42).reset_index(drop=True)

# 3. 모델 학습 (공부 시작)
vectorizer = CountVectorizer(max_features=2000)
X = vectorizer.fit_transform(df['text']).toarray()
y = torch.FloatTensor(df['label'].values).unsqueeze(1)

class ExtendedClassifier(nn.Module):
    def __init__(self, input_dim):
        super(ExtendedClassifier, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

model = ExtendedClassifier(X.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

model.train()
inputs = torch.FloatTensor(X)

print("🚀 모델이 500회 학습을 시작합니다! (잠시만 기다려주세요...)")

# [수정됨] 500회 반복 학습
for epoch in range(500):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    # 100번마다 진행상황 출력
    if (epoch+1) % 100 == 0:
        print(f"학습 진행률: {epoch+1}/500 (Loss: {loss.item():.6f})")

torch.save(model.state_dict(), 'model_artifacts/model_weights.pth')
joblib.dump(vectorizer, 'model_artifacts/vectorizer.pkl')
print("모델 학습 완료!\n")

# 4. 예측 테스트
def predict_politeness(sentences):
    model.eval()
    vectorizer = joblib.load('model_artifacts/vectorizer.pkl')

    test_inputs = vectorizer.transform(sentences).toarray()
    test_tensor = torch.FloatTensor(test_inputs)

    with torch.no_grad():
        outputs = model(test_tensor)
        probs = outputs.numpy().flatten()
        preds = (probs > 0.5).astype(int)

    print(f"{'입력 문장':<40} | {'판단 결과 (Label)'}")
    print("-" * 80)
    for text, pred in zip(sentences, preds):
        if pred == 1:
            result = "Label 1 : 격식 (교수님용)"
        else:
            result = "Label 0 : 비격식 (친구용)"

        print(f"{text:<40} | {result}")

my_sentences = [
    "교수님 안녕하세요, 과제 관련 문의드립니다.",
    "야 너 오늘 피방 갈래?",
    "혹시 랩미팅 일정이 변경되었나요?",
    "아 진짜 개피곤하다 ㅋㅋㅋ",
    "죄송하지만 출석 인정 서류 제출이 늦어질 것 같습니다."
]

predict_politeness(my_sentences)

🚀 모델이 500회 학습을 시작합니다! (잠시만 기다려주세요...)
학습 진행률: 100/500 (Loss: 0.000000)
학습 진행률: 200/500 (Loss: 0.000000)
학습 진행률: 300/500 (Loss: 0.000000)
학습 진행률: 400/500 (Loss: 0.000000)
학습 진행률: 500/500 (Loss: 0.000000)
모델 학습 완료!

입력 문장                                    | 판단 결과 (Label)
--------------------------------------------------------------------------------
교수님 안녕하세요, 과제 관련 문의드립니다.                 | Label 1 : 격식 (교수님용)
야 너 오늘 피방 갈래?                            | Label 0 : 비격식 (친구용)
혹시 랩미팅 일정이 변경되었나요?                       | Label 1 : 격식 (교수님용)
아 진짜 개피곤하다 ㅋㅋㅋ                           | Label 0 : 비격식 (친구용)
죄송하지만 출석 인정 서류 제출이 늦어질 것 같습니다.           | Label 1 : 격식 (교수님용)
